In [1]:
#=========================================================================================================
#================================ 0. MODULE


# Class
from numerai import Numerai

# Architecture
from architecture import models, nCores


#=========================================================================================================
#================================ 1. CLASS


stacking = Numerai(week=109)


#=========================================================================================================
#================================ 2. FEATURE ENGINEERING


# metafeature = ['variance', 'mean', 'distance']
# stages = [2, 0, 2]
# stacking.add_metafeature(metafeature, stages)


#=========================================================================================================
#================================ 3. TRAINING MODEL


###### MACHINE LEARNING
stacking.add_model(models)
stacking.fit_tune(nCores, stageNumber=1, neuralNetworkCompiler=False, evaluate=True, interaction=None)


---------------------------------------------
>> Loading data...done

Xtrain1: (157445, 50) 
Ytrain1: (157445,) 
Xtrain2: (158232, 50) 
Ytrain2: (158232,) 
Xtest: (77936, 50) 
Ytest: (77936,) 
Xvalid: (46362, 50) 
Yvalid: (46362,) 
Submit data: (243281, 50)


---------------------------------------------
>> Processing first stage

>> Processing LightGBM1 ------

Step 1...

KeyboardInterrupt: 

In [ ]:
###### DEEP LEARNING
stacking.neuralNetworkCompiler(learningRate=0.0001, batch=512, epoch=10, cvNumber=1, displayStep=500, evaluate=True, useGPU=True)

In [ ]:
#=========================================================================================================
#================================ 4. PREDICTION

# stacking.fit_tune(nCores, neuralNetworkCompiler=False, evaluate=False, interaction=None)
# stacking.neuralNetwork(learningRate=0.0001, batch=64, epoch=2, cvNumber=2, displayStep=10000, evaluate=False, useGPU=True)
# stacking.submit(5, submissionNumber=2, week=109)